In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Novel Generation with LLM
* Langchain API를 이용해서 소설을 작성해볼 예정입니다.
* 직접 프롬프트를 제작하고, 이를 Langchain api에 입력해 소설을 출력해봅니다.

In [2]:
!pip install openai langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the

In [3]:
from typing import Dict, List

from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from pydantic import BaseModel


### OpenAI API key
* key 입력하기

In [4]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [5]:
TMP_PATH = "/content/drive/MyDrive/dataset/Novel_generation/single_prompt/prompt_template.txt"

### Langchain prompt
* 유저 입력을 이용해서 prompt를 전달해봅시다.

In [7]:
class UserRequest(BaseModel):
    genre: str
    characters: List[Dict[str, str]] # 캐릭터 이름, 역할 (주연, 조연)
    idea: str


def read_prompt_template(file_path: str) -> str:
    with open(file_path, "r") as f:
        prompt_template = f.read()

    return prompt_template


def generate_novel(req: UserRequest) -> Dict[str, str]:
                        # temperature와 max_tokens은 임의 설정
    writer_llm = ChatOpenAI(temperature=0.1, max_tokens=500, model="gpt-3.5-turbo")
    writer_prompt_template = ChatPromptTemplate.from_template(
        template=read_prompt_template(TMP_PATH)
    )
    writer_chain = LLMChain(
        llm=writer_llm, prompt=writer_prompt_template, output_key="output"
    )

    result = writer_chain(req.dict())

    return {"results": result["output"]}

### Prompt 참고자료 작성
* txt 파일에 존재하는 템플릿 프롬프트를 동작시킬 프롬프트를 작성합니다.
* user_data에 의해 작성된 데이터는 이미 생성된 템플릿 프롬프트에 추가되어 처리됩니다.

In [14]:
user_data = {
    "genre": "액션",
    "characters": [
        {
            "name": "김철수",
            "role": "주인공"
        },
        {
            "name": "이영희",
            "role": "조연"
        }
    ],
    "idea": "날씨가 추워지고 있습니다."
}


In [15]:
request_instance = UserRequest(**user_data)

In [16]:
generate_novel(request_instance)

{'results': '제목: 얼음의 전사\n\n줄거리:\n김철수는 얼음의 힘을 가진 주인공으로, 얼음 왕국을 지키는 임무를 맡고 있다. 얼음 왕국은 평화롭게 지내던 중, 이영희라는 조연이 얼음 왕국에 도착한다. 그녀는 얼음 왕국의 비밀을 알고 있는 인물로, 얼음 왕국을 파괴하려는 악당들의 손에 빠져있었다. 김철수는 이영희를 구출하고, 함께 얼음 왕국을 지키기 위해 모험을 떠난다.\n\n김철수와 이영희는 얼음의 힘을 이용하여 적들과 맞서 싸우며, 다양한 액션 장면을 펼친다. 얼음의 힘을 마스터한 김철수는 얼음 화살로 적들을 공격하고, 얼음 방패로 자신과 이영희를 보호한다. 그리고 얼음 왕국의 비밀을 알아가면서, 악당들의 음모를 파헤치고 얼음 왕국을 구하기 위해 사투를 벌인다.\n\n김철수와 이영희는 서로의 신뢰를 강화시키며, 얼음 왕국을 지키기 위해 힘을 합친다. 그들은 얼음 왕국의 마지막 결전지로 향하고, 악당들과의 대결에서 승리를 거둔다. 얼음 왕국은 평화로운'}